In [1]:
import numpy as np
from scipy.io import loadmat
import matplotlib
matplotlib.use("TkAgg")
import matplotlib.pyplot as plt
import pickle

In [2]:
# data = loadmat('uft_flight07.mat', variable_names=('lowT_av', 'upT_av', 'lwc1V_av'))

# with open("data.pickle", "wb") as f:
#     pickle.dump(data, f)

with open("data.pickle", "rb") as f:
    data = pickle.load(f)

In [3]:
LOW =  275000
HIGH = 300000

Y1 = data['lowT_av'].squeeze()
Y2 = data['upT_av'].squeeze()
LWC = data['lwc1V_av']
X = np.arange(Y1.shape[0]) / 100.

X_cut = X[LOW:HIGH]
Y1_cut = Y1[LOW:HIGH]
Y2_cut = Y2[LOW:HIGH]

In [4]:
def rolling_window(a, window):
    shape = a.shape[:-1] + (a.shape[-1] - window + 1, window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

def detect_jumps(data, size, threshold, func=lambda x: x.max(1) - x.min(1)):
    assert size % 2 # Only odd sizes, for now
    pad = size // 2
    
    strided_data = rolling_window(data, size)
    #diff = strided_data.max(axis=1) - strided_data.min(axis=1)
    result = func(strided_data)
    result = np.hstack((np.zeros(pad), result, np.zeros(pad)))
    return result > threshold

In [ ]:
plt.rcParams['figure.figsize'] = (18,4)


WIDTH = 0.5


jumps = detect_jumps(Y1_cut, 5, .01)

highlight = np.where(jumps)[0] / 100.
#highlight /= (HIGH - LOW) / 100.
highlight += LOW / 100.


plt.vlines(highlight, 13.8, 14.6, alpha=.25, linewidth=2.5, colors='r')
plt.plot(X_cut, Y1_cut, linewidth=WIDTH)
#plt.plot(X_cut, Y2_cut, linewidth=WIDTH)
#plt.plot(X[LOW:HIGH], 20*LWC[LOW:HIGH])
plt.show()

# To do:

0. Estimate the optimal threshold value, maybe via a purely statistical analysis (i.e. fraction of all points that are marked as jumps)

0. Estimate the optimal window (other than 5 - maybe)

0. Try to make it more robust to excessive noise (divide the diff term by std?)

0. Substract linear baseline

0. More sophisticated algorithm?